## 1. Базовые подходы к обработке текста

В области обучения с учителем, например, для выполнения задач классификации, обычно наша цель - найти параметризованную модель, лучшую в своем классе.: <br><br> $$A(X, \hat{w}): A(X, \hat{w}) \simeq f(X) \Leftrightarrow A(X, \hat{w}) = \operatorname*{arg\,min}_w \left\|A(X, w) - f(X)\right\|$$

Где $X \in R^{ n\times m}$ - матрица фичей ($n$ наблюдений с $m$ фичами), $w \in R^{m}$ - вектор параметров модели, $\hat{w}$ - "лучшие" параметры модели

Однако, как кандидат для X - все, что у нас есть - <strong>это сырой текст, алгоритмы не могут использовать его в таком виде.</strong>

Чтобы применить машинное обучение к текстовым данным, нам сначала нужно преобразовать такой контент в некий числовой формат (чтобы сформировать векторы признаков). 

В обработке естественного языка автоматическое извлечение признаков может быть достигнуто разными способами: <strong>(bag-of-words, word embeddings, graph-based representations, etc.)</strong>

Сегодня мы углубимся в детали подхода и методов <strong> bag-of-words </strong>, построенных на его основе в библиотеке Scikit-Learn.

## 2. Bag-of-Words Approach

### 2.1 Intuition Behind the Model. Word Counters.

В рамках подхода bag-of-words мы работаем при следующих предположениях:
* Текст можно анализировать без учета порядка слов / токенов
* Нам нужно только знать, из каких слов / токенов состоит текст и сколько раз мы их встречали
* Чем чаще слово / токен появляется в документе, тем он важнее
Более формально, представленный набор текстов $T_1, T_2, ... , T_n$, мы извлекаем уникальные токены $w_1, w_2, ..., w_m$ чтобы сформировать словарь.

Поэтому каждый текст $T_i$ представлен вектором фич $F_j = \{x_{ij},\ j \in [1,m]\}$, где $x_{ij}$ соотвествует числу соовстречаний со словом $w_j$ в тексте $T_i$

Например, наш корпус состоит из **2 текстов**:

["I love data science", 
"A data scientist is often smarter than a data analyst"]

\* **На этапе предварительной обработки все буквы обычно приводят к нижнему регистру, иногда выполняется стемминг / лемматизация, а также удаление стоп-слов / знаков препинания, но это не обязательно.**

Предположим, наши токены - простые униграммы (слова), поэтому мы можем выделить **11 уникальных слов**: : {i, love, data, science, a, scientist, is, often, smarter, than, analyst}

Затем наш корпус сопоставляется с векторами признаков. $T_1=(1,1,1,1,0,0,0,0,0,0,0)$, $T_2=(0,0,2,0,2,1,1,1,1,1,1)$

|Text #|i|love|data|science|a|scientist|is  |often|smarter|than|analyst|
|------|------|------|------|------|------|------|------|------|------|------|------|
|$T_1$|1|1|1|1|0|0|0|0|0|0|0|
|$T_2$|0|0|2|0|2|1|1|1|1|1|1|

Насколько эффективен этот подход с точки зрения памяти?
Если n == 20k, этот текстовый корпус может породить словарь примерно с 100k элементами.
<br> Таким образом, для хранения X в виде массива типа int32 потребуется 20000 x 100000 x 4 байта ~ **8 ГБ ОЗУ**, что с трудом возможно на современных компьютерах.
К счастью, **большинство значений в X будут нулями**, поскольку для данного документа будет использовано менее пары тысяч (или даже сотен) отдельных слов. По этой причине мы говорим, что токены слов **обычно являются разреженными наборами данных большой размерности**. Мы можем сэкономить много памяти, сохраняя в памяти только ненулевые части векторов признаков.
Разреженные матрицы - это структуры данных, которые делают именно это, и scikit-learn имеет встроенную поддержку этих структур.

#### Преимущества
* Очень интуитивно понятный подход, простой в использовании, понимании и применении - вы можете сами запрограммировать
* Встроенная поддержка многих библиотек.
* Разреженный формат с эффективным использованием памяти, приемлемый для большинства алгоритмов
* Несмотря на простоту, работает хорошо, можно добиться хороших результатов
* Быстрая предварительная обработка даже на 1 ядре

#### Недостатки
* Огромный корпус обычно приводит к огромному объему словарного запаса (миллионы слов), даже разряженный формат вам не поможет (только уловки хеширования)
* Существуют другие подходы, позволяющие уловить больше деталей (семантика, отношения, структура) - вложения слов и т.д.
* bag of words - это беспорядочное представление: отказ от пространственных отношений между функциями приводит к тому, что упрощенная модель не позволяет нам различать предложения, построенные из одних и тех же слов, но имеющие противоположные значения:
    * "New episodes **don't** feel like the first - watch it!" (positive)
    * "New episodes feel like the first - **don't** watch it!" (negative)
* **Однако это можно отчасти исправить увеличением «длины» токена (unigrams $\rightarrow$ bigrams, n-grams etc.), склеиванием отрицательных частиц со следующим словом (not like $\rightarrow$ not_like), использованием character n-grams, skip-grams etc.** (смотрите [эту секцию для лучшего понимания n-gram](#3_5))

### 2.2 Улавливание зависимостей. N-grams

Модель Bag-of-Words (BoW), построенная на простых токенах (униграммах), слишком упрощена и не улавливает пространственных зависимостей.
Чтобы разобраться с этим и расширить наши знания, давайте вкратце вспомним, что такое **N-gram**:
* N-gram это последовательность из $N$ токенов. 
* N-grams могут быть определенны по-разному, зависит от определения токенов. ('word', 'character', 'character_wb' etc.)

1) **Word n-grams: (уловить больше семантики)** 
* unigrams: 'I love data science' $\rightarrow$ [i, love, data, science]
* bigrams (2-grams): 'I love data science' $\rightarrow$ [i love, love data, data science]
* 3-grams: 'I love data science' $\rightarrow$ [i love data, love data science]
* ...

2) **Character n-grams: (позволяет ловить такие фичи, как ":)", отчасти разобраться со словами с ошибками, такими как "looong" etc.)**
* 5-grams: 'I love data science' $\rightarrow$ ["i lov", " love", ... , "cienc", "ience"]
* ...

3) **Character-wb n-grams (n-grams, только в рамках слова):**
* 5-grams: 'I love data science' $\rightarrow$ {" i ", " love", "love ", ... , "cienc", "ience"]
* ...

4) **Skip-n-grams or k-skip-n-grams (оба character- и word-based, расширяет использование зависимостей)**
* Последовательность из $N$ базовых токенов, имея расстояние из $\leq K$ токенов между ними
* 1-skip-2-grams: 'I love data science' $\rightarrow$ [i data, love science]
* ...
 



#### Преимущества

Тоже самое что и Bag-of-Words + улавливает больше контекста

#### Недостатки

Не забывайте, что с увеличением диапазона n-gram словарный запас **быстро растет**!
<br>**|(1,1)-grams| << |(1,2)-grams| << |(1,3)-grams| << ...**
<br>where (1,1)-grams = unigrams, (1,2)-grams = unigrams AND bigrams, etc.

### 2.3  CountVectorizer

CountVectorizer в Sklearn реализует вышеупомянутый подход Bag-of-Words:

**Часто используемые параметры:**
* **analyzer**={‘word’, ‘char’, ‘char_wb’} - какой вид токенов использовать (word, char-n-grams etc.)
* **ngram_range**=(min_n, max_n) - какое число токенов объединять в н-граммы. Например, ngram_range=(1,2) $\rightarrow$ использование unigrams и bigrams
* **stop_words**={‘english’, list_of_words, or None} - какие стоп-слова фильтровать и нужно ли это делать
* **vocabulary**={None, your_own_dictionary} - использовать ли данный словарь или строить его из извлеченных токенов
* **max_features**={N, None} - для создания словаря, который рассматривает **топ-N** терминов, упорядоченных по частоте терминов (TF) в корпусе
* **max_df** – при построении словаря игнорировать термины, частота которых в документе строго превышает заданный порог (стоп-слова, специфичные для корпуса)
* **min_df** – при построении словаря игнорировать термины, частота которых в документе строго ниже заданного порога

In [29]:
# usage example

# import CountVectorizer from sklearn library
from sklearn.feature_extraction.text import CountVectorizer

# create CountVectorizer object
cv = CountVectorizer(
                    analyzer='word', # token = word
                    ngram_range=(1,1), # only unigrams are used, (1,2) - unigrams/bigrams, ..., etc.
                    stop_words=['my', 'stop', 'word', 'list'], # or stop_words='english'
                    vocabulary=None, # or vocabulary=your_own_dictionary
                    max_df=1.0, # don't filter words by their frequency
                    max_features=6 # only top-6 words will be used as columns
                    )

In [30]:
# Мы будем использовать его в качестве примера для других методов извлечения признаков.
# В качестве входящих данных вы можете использовать списки, массивы numpy, pandas DataFrames.
texts = [
    'nobody can stop me', # "stop" will be filtered by stop_words list
    'word is a building blocks of a text', # "word" will be filtered by stop_words list
    'I like doing feature extraction on text',
    'I do not like digits in text like'
    ]

In [31]:
# применение CountVectorizer на текстовом корпусе
transformed_texts_cv = cv.fit_transform(texts)
# конвертация разряженной репрезентации в dense формат для исследования
print('Obtained feature matrix X:')
print(transformed_texts_cv.todense(), '\n')

Obtained feature matrix X:
[[0 1 1 0 0 0]
 [0 0 0 0 1 1]
 [1 0 0 0 0 1]
 [2 0 0 1 0 1]] 



In [32]:
# отобразить словарь (отсортированный по индексу колонки), чтобы увидить сопоставление индексов / столбцов и слов 
print('Dictionary:')
for k,v in sorted(cv.vocabulary_.items(), reverse=False):
    print('column index:{}, token: {}'.format(v,k))

Dictionary:
column index:0, token: like
column index:1, token: me
column index:2, token: nobody
column index:3, token: not
column index:4, token: of
column index:5, token: text


In [33]:
# преобразовать новые предложения (having CountVectorizer trained)
new_text = ['i like feature extraction very much'] 
new_transformed = cv.transform(new_text)
# некоторые слова, как "very" и "much", не использовались при построение словаря, поэтому они были пропущенны
print('\nNew sentence (transformed):')
print(new_transformed.todense(), '\n')


New sentence (transformed):
[[1 0 0 0 0 0]] 



### 2.4 TF-IDF. TfIdfVectorizer

В TF-IDF подходе (term frequency - inverse document frequency), в дополнение к обычной BoW-модели сделаны следующие дополнения:
* Текст можно анализировать без учета порядка слов/токенов
* Нам нужно только знать, из каких слов/токенов состоит текст и сколько раз мы их встречали
* Чем чаще слово/токен появляется в документе, тем он важнее
* **Если слово/токен появляется в документе, но редко встречается в других документах - это важно, и наоборот: <br>если оно часто встречается в большинстве документов - тогда мы не можем полагаться на это слово, чтобы помочь нам различать тексты**
Таким образом, мы смотрим на весь корпус, обычные частоты слов (частоты терминов, TF) взвешиваются по множителю IDF:

$$  
    \begin{cases} TF(w,T)=n_{Tw} \\ IDF(w, T)= log{\frac{N}{n_{w}}}\end{cases} \implies 
    TF\text{-}IDF(w, T) = n_{Tw}\ log{\frac{N}{n_{w}}} \ \ \ \ \forall w \in W
$$

<br> где $T$ соответствует количество документов (text), 
<br>$w$ - выбранное слово в документе T, 
<br>$n_{Tw}$ - число использований слова $w$ в тексте $T$, 
<br>$n_{w}$ - число документов содержащих слово $w$, 
<br> $N$ - общее число документов в корпусе.


**Часто используемые параметры:**
* **analyzer**={‘word’, ‘char’, ‘char_wb’} - какой вид токенов использовать (word, char-n-grams etc.)
* **ngram_range**=(min_n, max_n) - какое число токенов объединять в н-граммы. Например, ngram_range=(1,2) $\rightarrow$ использование unigrams и bigrams
* **stop_words**={‘english’, list_of_words, or None} (default) - какие стоп-слова фильтровать и нужно ли это делать
* **vocabulary**={None, your_own_dictionary} - использовать ли данный словарь или строить его из извлеченных токенов
* **max_features**={N, None} - для создания словаря, который рассматривает **топ-N** терминов, упорядоченных по частоте терминов (TF) в корпусе
* **norm**={‘l1’, ‘l2’ or None, optional} - норма для регуляризации ($L_2-$, $L_1-$ norms)
* **smooth_idf**={True, False} Сглаживает веса idf, добавляя единицу к частоте документов, как если бы был замечен дополнительный документ, содержащий каждый термин в коллекции ровно один раз. Предотвращает нулевые деления.
* **max_df** – при построении словаря игнорировать термины, частота которых в документе строго превышает заданный порог (стоп-слова, специфичные для корпуса)
* **min_df** – при построении словаря игнорировать термины, частота которых в документе строго ниже заданного порога

In [ ]:
# usage example

# import TfidfVectorizer from sklearn library
from sklearn.feature_extraction.text import TfidfVectorizer

# create TfidfVectorizer object
tv = TfidfVectorizer(
                    analyzer='word', # token = word
                    ngram_range=(1,1), # only unigrams are used, (1,2) - unigrams/bigrams, ..., etc.
                    stop_words=['my', 'stop', 'word', 'list'], # or stop_words='english'
                    vocabulary=None, # or vocabulary=your_own_dictionary
                    max_df=1.0, # don't filter words by their frequency
                    max_features=6, # only top-6 words will be used as columns,
                    smooth_idf=True,
                    norm='l2' # euclidean norm используется по дефолту
                    )

In [ ]:
# применяем TfidfVectorizer к нашему корпусу
transformed_texts_tv = tv.fit_transform(texts)
# конвертация разряженной репрезентации в dense формат для исследования
print('Obtained feature matrix X (see, L2-norm is used):')
print(transformed_texts_tv.todense(), '\n')

In [ ]:
# отобразить словарь (отсортированный по индексу колонки), чтобы увидить сопоставление индексов / столбцов и слов 
print('Dictionary:')
for k,v in sorted(tv.vocabulary_.items(), reverse=False):
    print('column index:{}, token: {}'.format(v,k))

In [ ]:
# преобразовать новые предложения (having CountVectorizer trained)
new_text = ['i like extraction very much'] 
new_transformed = tv.transform(new_text)
# некоторые слова, как "very" и "much", не использовались при построение словаря, поэтому они были пропущенны
print('\nNew sentence (transformed):')
print(new_transformed.todense(), '\n')

### 2.5 Hashes. HashingVectorizer

Хеш-функция - это любая функция, которая **может использоваться для сопоставления данных произвольного размера с данными фиксированного размера**. 
<br>Значения, возвращаемые хеш-функцией, называются хеш-значениями, хеш-кодами или просто хешами.
<br>$f(X) \rightarrow \{0,N-1\}:\ f(X) = X\  mod\ N$, функция, которая отображает входящие данные в набор из $N$ "корзин", является примером хеш-функции:


Например, $N = 2^k = 2^3 = 8$, then $\ f(15)=15\ mod \ 8 = 7,\ f(9)=9\ mod \ 8 = 1,\ ...$

В этой реализации векторизатора используется трюк с хешированием, чтобы найти сопоставление **имени строки токена** с **целочисленным индексом** функции.

#### Преимущества:

*  **Очень масштабируемая память для больших наборов данных**, поскольку нет необходимости хранить словарь в памяти
* Быстро сериализовать/десериализовать, поскольку он не содержит состояния, кроме параметров конструктора
* Может использоваться в потоковой передаче (частичное хэширования) и/или распараллеливаться, поскольку во время хэширования состояние не вычисляется
* Может использоваться как «глупое» уменьшение размерности

#### Недостатвки (vs Vectorizers со словарем в памяти): 

* Невозможно вычислить обратное преобразование (чтобы перейти от индексов функций к строковым именам функций), <br>которые **могут быть проблемой при попытках анализа, какие фичи наиболее важны для модели**.
* Могут быть **коллизии**: разные токены могут быть сопоставлены с одним и тем же «bucket» (индекс фичи). 
<br>Однако на практике это редко является проблемой, если количество bucket'ов достаточно велико. (e.g. $2^{18}$ для задач текстовой классификации)


\* Используемая хеш-функция - это подписанная 32-разрядная версия Murmurhash3 (для тех кто в этом заинтересован :)  )

**Часто используемые параметры:**
* **analyzer**={‘word’, ‘char’, ‘char_wb’} - какой вид токенов использовать (word, char-n-grams etc.)
* **ngram_range**=(min_n, max_n) - какое число токенов объединять в н-граммы. Например, ngram_range=(1,2) $\rightarrow$ использование unigrams и bigrams
* **stop_words**={‘english’, list_of_words, or None} (default) - какие стоп-слова фильтровать и нужно ли это делать
* **n_features**={N} - сколько bucket'ов использовать
* **norm**={‘l1’, ‘l2’ or None, optional} - норма для регуляризации ($L_2-$, $L_1-$ norms)

In [ ]:
# пример использования

# import HashingVectorizer from sklearn library
from sklearn.feature_extraction.text import HashingVectorizer

# create HashingVectorizer object
hv = HashingVectorizer(
                    analyzer='word', # token = word
                    ngram_range=(1,1), # only unigrams are used, (1,2) - unigrams/bigrams, ..., etc.
                    stop_words=['my', 'stop', 'word', 'list'], # or stop_words='english'
                    n_features=6, # only 6 bins will be used as columns, high probability of collisions!
                    norm=None
                    )

In [ ]:
# применение HashingVectorizer на нашем корпусе
transformed_texts_hv = hv.fit_transform(texts)
# конвертация разряженной репрезентации в dense формат для исследования
print('Obtained feature matrix X (see, no norm is used):')
print(transformed_texts_hv.todense(), '\n')

In [ ]:
# I see no dictionary ...
print('Dictionary:')
print('Oops, Hashing trick assumes no vocabulary will be used at all, online learning :)')
print("However, we won't be able to do reverse transform and to get exact words :( ")

In [ ]:
  
# трансформация новых предложений (имея натренированный HashingVectorizer)
new_text = ['i like extraction very much'] 
new_transformed = hv.transform(new_text)
# некоторые слова, как "very" и "much", не использовались при построение словаря, поэтому они были пропущенны
print('\nNew sentence (transformed):')
print(new_transformed.todense(), '\n')

## 3. Going Beyond: Feature Engineering

Обычно конкретный домен ведет к определенной информации, скрытой внутри ваших данных.
Вам нужно извлечь как можно больше таких инсайтов.

Например, если мы хотим натренировать модель по сентимент анализу (задачу классификации) на IMDB датасете (обзоры фильмов), и нам кажется, что **многие обзоры могут содержать явные пометки (скажем, в формате x/xx)**, поэтому мы должны это проверить и извлечь полезную настраиваемую функцию:

["Average film, however, starring Matt Damon, 8/10", 1] $\rightarrow$ {"8/10"} $\rightarrow$ 8/10=0.8 ~ 1 $\rightarrow$ review is positive
<br>["2/10, there is nothing to add", 0] $\rightarrow$ {"2/10"} $\rightarrow$ 2/10=0.2 ~ 0 $\rightarrow$ review is negative.

Однако имейте в виду даты и выбросы (в отношении этой конкретной фичи) или что-то еще - всегда проверяйте свой код/регулярные выражения:

Например, некоректный парсинг **'01/10/1999'** приведет к **{1/10, 10/1999} или {1/10}  ~ 0 (негативный отзыв?!)** ошибкам.

### В дальнейшем мы обсудим особенности предметной области, но они не являются панацеей в целом.

### 3.1 Token-based Level

Нам нужно смотреть на токены (слова, сущности такие как улыбки и т.д.) и пытаться извлечь значимые фичи

* позитивные смайлики
* негативные смайлики
* явная оценка

In [ ]:
import pandas as pd
import numpy as np
from textblob import Word, TextBlob
import re # for regular expressions

# download resources to be used by TextBlob wrapper (if not yet downloaded)
import nltk
nltk.download('punkt')
pass

In [ ]:
# this implemenation does not deal with aforementioned cases, 
# to extract rating "candidates" in a text s
def get_rate(s):
    # searching for possible candidates
    candidates = re.findall(r'(\d{1,3}[\\|/]{1}\d{1,2})', s)
    rates = []
    for c in candidates:
        try:
            rates.append(eval(c)) # by the way, "eval" is a prime evil, it may lead you to the dark side :)
            # instead, say, install sympy
            # from sympy import sympify
            # sympify("1*5/6*(7+8)").evalf()
        except SyntaxError:
            pass
        except ZeroDivisionError:
            return 0
    return np.mean(rates) if rates else -1 # if there is more than 1 value, calculate mean

# bags of positive/negative smiles
positive_smiles = set([
":‑)",":)",":-]",":]",":-3",":3",":->",":>","8-)","8)",":-}",":}",":o)",":c)",":^)","=]","=)",":‑D",":D","8‑D","8D",
"x‑D","xD","X‑D","XD","=D","=3","B^D",":-))",";‑)",";)","*-)","*)",";‑]",";]",";^)",":‑,",";D",":‑P",":P","X‑P","XP",
"x‑p","xp",":‑p",":p",":‑Þ",":Þ",":‑þ",":þ",":‑b",":b","d:","=p",">:P", ":'‑)", ":')",  ":-*", ":*", ":×"
])
negative_smiles = set([
":‑(",":(",":‑c",":c",":‑<",":<",":‑[",":[",":-||",">:[",":{",":@",">:(","D‑':","D:<","D:","D8","D;","D=","DX",":‑/",
":/",":‑.",'>:\\', ">:/", ":\\", "=/" ,"=\\", ":L", "=L",":S",":‑|",":|","|‑O","<:‑|"
])

# function to extract token-level features from texts
def get_token_level_features(texts, visualize=True):
    
    # assume texts = pd.Series with review text
    print('extracting token-level features...')
    tdf = pd.DataFrame()
    tdf['text'] = texts # this is our review
    
    # 1. extract rating, like "great film. 9/10" will yield 0.9
    tdf['rating'] = tdf['text'].apply(get_rate).fillna(-1) # rating (if found in review, else substitute NaN's by -1)

    # 2. extract smiles and count positive/negative smiles per review
    tdf['positive_smiles'] = tdf.text.apply(lambda s: len([x for x in s.split() if x in positive_smiles]))
    tdf['negative_smiles'] = tdf.text.apply(lambda s: len([x for x in s.split() if x in negative_smiles]))
    
    if visualize:
        # this is used for visual clarity, return pd.DataFrame
        return tdf 
    else:
        # get correct (and sparse) representation of feature matrix F
        from scipy.sparse import csr_matrix 
        return csr_matrix(tdf[tdf.columns[1:]].values)

### 3.2 Sentence-based / Text-based Level

Мы перешли на уровень предложения/текста.
<br> Давайте посмотрим, какие фичи мы можем искать:
* **Количество предложений** (текст нужно разбить на предложения, затем извлечь длину полученного списка)
* **Количество восклицательных знаков** (целое число) или присутствие (boolean) - улавливание стресса, особенно если мы используем вероятностный вывод вместо бинарной классификации
* **Подсчет вопросительных знаков** (целое число) или присутствие (boolean) - иногда помогает уловить сарказм.
* **Количество слов в верхнем регистре** (длины > 1, чтобы исключить начало предложений) - напряжение текста, особенно если мы используем вероятностный вывод вместо бинарной классификации
* **Контрастные спряжения**, например {'вместо', 'тем не менее', 'наоборот', 'с другой стороны'} - чтобы уловить возможные изменения настроения.

Немного информации о текстовых "краях" - первое / последнее предложения в обзоре:
* **"полярность" первого/последнего продложения**
* **"субъективность" первого/последнего продложения**
* **"чистота" первого/последнего продложения или всего набора предложений** - поймать изменения тональности

In [ ]:
# let's continue...

# контрастные соединения
contrast_conj = set([
'alternatively','anyway','but','by contrast','differ from','elsewhere','even so','however','in contrast','in fact',
'in other respects','in spite of','in that respect','instead','nevertheless','on the contrary','on the other hand',
'rather','though','whereas','yet'])

# чтобы получить "чистоту" отзыва ~ показывает ту же тональность к обзору (~ 1) или изменение настроения (~ 0)
def purity(sentences):
    # получает полярность по всем предложениям
    polarities = np.array([TextBlob(x).sentiment.polarity for x in sentences])
    return polarities.sum() / np.abs(polarities).sum()

# uppercase pattern
uppercase_pattern = re.compile(r'(\b[0-9]*[A-Z]+[0-9]*[A-Z]+[0-9]*\b)')

# регулярное выражение для разделения отзыва на предложения, вы можете использовать метод из textblob: TextBlob(x).sentences_
sentence_splitter = re.compile('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<![A-Z]\.)(?<=\!|\?|\.)\s')
# you can https://regex101.com/ for regex creation/checking, very convenient

# feature engineering
def get_text_level_features(texts, visualize=True):
    # assume text = pd.Series with review text
    print('extracting text-level features...')
    tdf = pd.DataFrame()
    tdf['text'] = texts # наш отзыв
    tdf['sentences'] = tdf.text.apply(lambda s: re.split(sentence_splitter, s)) # разделяем на предложения
    
    tdf['sentence_cnt'] = tdf['sentences'].apply(len) # считаем предложения
    tdf['exclamation_cnt'] = tdf.text.str.count('\!') # считаем знаки восклицания
    tdf['question_cnt'] = tdf.text.str.count('\?') # считаем вопросильные знаки
    
    # uppercase words cnt (like HOLY JESUS!)
    tdf['upper_word_cnt'] = tdf.text.apply(lambda s: len(re.findall(uppercase_pattern, s)))
    
    # contrast conjugations
    tdf['contrast_conj_cnt'] = tdf.text.apply(lambda s: len([c for c in contrast_conj if c in s]))
    
    # polarity of 1st sentence
    tdf['polarity_1st_sent'] = tdf.sentences.apply(lambda s: TextBlob(s[0]).sentiment.polarity)
    # subjectivity of 1st sentence
    tdf['subjectivity_1st_sent'] = tdf.sentences.apply(lambda s: TextBlob(s[0]).sentiment.subjectivity)
    # polarity of last sentence
    tdf['polarity_last_sent'] = tdf.sentences.apply(lambda s: TextBlob(s[-1]).sentiment.polarity)
    # subjectivity of last sentence
    tdf['subjectivity_last_sent'] = tdf.sentences.apply(lambda s: TextBlob(s[-1]).sentiment.subjectivity)
    # subjectivity of review itself
    tdf['polarity'] = tdf.text.apply(lambda s: TextBlob(s[-1]).sentiment.polarity)
    # "purity" of review, |sum(sentence polarity) / sum(|sentence polarity|))|, ~ 1 is better, ~ 0 -> mixed
    tdf['purity'] = tdf.sentences.apply(purity)
    tdf['purity'].fillna(0, inplace=True)
    
    if visualize:
        # для визуализации dataframe
        return tdf 
    else:
        # для разряженного представления
        from scipy.sparse import csr_matrix 
        return csr_matrix(tdf[tdf.columns[2:]].values)

### БУДЬТЕ ОСТОРОЖНЫ, если вы используете ЛИНЕЙНЫЕ МОДЕЛИ и имеете ЕЩЕ КОРОТКИЕ ОТЗЫВЫ (1 предложение),
### tdf['subjectivity_1st_sent'] ~ tdf['subjectivity_last_sent'], то получите одинаковые значения, что приведет к неифнормативности фичи

In [ ]:
# let's test custom features:

reviews = [
    "Waste of time :( 2/10 for the plot and 4/10 for acting!",
    'Awful film! Nobody can like it',
    'Wow! Am I impressed?? TOTALLY :D',
    '7/10'
]

# token-based
token_lf = get_token_level_features(reviews)
token_lf

In [ ]:
# token-based
token_lf = get_text_level_features(reviews)
token_lf

# Пример 

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from collections import Counter

In [ ]:
train = fetch_20newsgroups()
test = fetch_20newsgroups(subset="test")

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('clf', LogisticRegression()),
])

In [ ]:
params = dict(clf__C=[10, 1, 0.1, 0.01])
grid_search = GridSearchCV(pipeline, params, scoring="accuracy", cv=skf, n_jobs=-1)

In [ ]:
grid_search.fit(train["data"], train["target"], )

In [ ]:
grid_search.best_score_, grid_search.best_estimator_

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('clf', LogisticRegression(C=1)),
])
pipeline.fit(train["data"], train["target"])

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
predictions = pipeline.predict(test["data"])
accuracy_score(test["target"], predictions)

In [ ]:
print(classification_report(test["target"], predictions, target_names=test["target_names"]))